In [1]:
using Revise
using MeshCat
using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using MeshCatMechanisms
using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
vis = Visualizer()
open(vis)

Listening on 0.0.0.0:8700...


Info: Serving MeshCat visualizer at http://127.0.0.1:8700


In [3]:
urdf = joinpath("..", "urdf", "panda", "panda_arm_box.urdf")
mechanism = parse_urdf(Float64, urdf)
box = findbody(mechanism, "box")
basejoint = joint_to_parent(box, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf), vis);

In [4]:
function make_end_effector_controller(state::MechanismState,
                                      end_effector::RigidBody,
                                      end_effector_point::Point3D,
                                      desired_point::Point3D)
    
    mechanism = state.mechanism
    v̇ = similar(velocity(state))
    
    joint_path = path(mechanism, root_body(mechanism), end_effector) # from root to end effector
    Jp = point_jacobian(state, joint_path, transform(state, end_effector_point, root_frame(mechanism)))
        
    function controller!(τ, t, state)
        ef_point_in_world = transform_to_root(state, end_effector) * end_effector_point
        point_jacobian!(Jp, state, joint_path, ef_point_in_world)
        des_point_in_world = transform(state, desired_point, root_frame(mechanism))
        Kp = 10.
        Kd = 5.
        Δp = des_point_in_world - ef_point_in_world
        v̇ .= Kp * Array(Jp)' * Δp.v .- Kd .* velocity(state)
        τ .= inverse_dynamics(state, v̇)
    end
end

state = MechanismState(mechanism)
desired_point_v = [1., 0., .25]
end_effector = findbody(mechanism, "panda_link8")
end_effector_point = Point3D(default_frame(end_effector), [0.,0.,0.])
desired_point = Point3D(root_frame(mechanism), desired_point_v)
point_ctrl! = make_end_effector_controller(state, end_effector, end_effector_point, desired_point)

LoadError: [91mUndefVarError: state not defined[39m

Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [ ]:
# q0 = [1.,0.,0.,0.,0.,0.,0.,-0.00159265,
#       0.858407,0.0184073,-1.89159,0.0884073,
#       -1.94159,0.138407]
q0 = [-0.00159265,
      0.858407,0.0184073,-1.89159,0.0884073,
      -1.94159,0.138407]
set_configuration!(state, q0)
t, q, v = simulate(state, 1.0);

In [ ]:
N = 200
x0 = MechanismState(mechanism)
Δt = 0.005
env = parse_contacts(mechanism, urdf, .5)
traj_exp = BilevelTrajOpt.simulate(x0,env,Δt,N,implicit_contact=false);
traj = traj_exp
q = [traj[1:num_positions(x0),i] for i in 1:N]
t = cumsum([Δt for i in 1:N]);

In [ ]:
setanimation!(mvis, t, q)

In [ ]:
widget = manipulate!(mvis)